# Rips Filtration

In this notebook, we introduce how to
- compute persistent homology in a stanadard way
- using enclosing radius optimization to save time
- update persistence (compute PH with warm start). 

In [1]:
import bats
import numpy as np
import scipy.spatial.distance as distance
import matplotlib.pyplot as plt
import time

In [2]:
# generate a random data set
n = 100
p = 2
data = np.random.rand(n, p)
plt.scatter(data[:,0], data[:,1])

In [3]:
# compute pairwise distances
D = distance.squareform(distance.pdist(data))

# Rips complex for full metric space
t0 = time.monotonic()
F = bats.LightRipsFiltration(bats.Matrix(D), np.inf, 2)
t1 = time.monotonic()
print("construction time: {} sec.".format(t1-t0))

# compute with F2 coefficents
t0 = time.monotonic()
R = bats.reduce(F, bats.F2())
t1 = time.monotonic()
print("reduction time: {} sec.".format(t1-t0))

construction time: 0.04950962300000006 sec.
reduction time: 0.582515844 sec.


### Enclosing radius
Now, we compute the enclosing radius of the finite metric space, which will be efficient to compute the homology of Rips Complex.

**Math explanation**:

The reason behind it is because at enclosing radius, all vertices are connected to an apex, which makes the space contractible and so with zero reduced homology.


In [4]:
# Two ways to find Enclosing Radius
r_enc = np.min(np.max(D, axis=0))
print("enclosing radius = {}".format(r_enc))

r_enc = bats.enclosing_radius(bats.Matrix(D))
print("enclosing radius = {}".format(r_enc))

enclosing radius = 0.6714091804599316
enclosing radius = 0.6714091804599316


In [5]:
# Rips complex up to enclosing radius
t0 = time.monotonic()
F_enc = bats.LightRipsFiltration(bats.Matrix(D), r_enc, 2)
t1 = time.monotonic()
print("construction time: {} sec.".format(t1-t0))

# compute with F2 coefficents
t0 = time.monotonic()
R_enc = bats.reduce(F_enc, bats.F2())
t1 = time.monotonic()
print("reduction time: {} sec.".format(t1-t0))

construction time: 0.02777356299999978 sec.
reduction time: 0.21394023200000012 sec.


Let's compare the number of simplices in each filtration

In [6]:
print("dim\tinf\tr_enc")
for k in range(F.maxdim() + 1):
    print("{}\t{}\t{}".format(k, F.ncells(k), F_enc.ncells(k)))
print("--------------------------")
# print("\t{}\t{}".format(F.ncells(), F2.ncells()))

dim	inf	r_enc
0	100	100
1	4950	3523
2	161700	69831
--------------------------


The savings are primarily in the number of 2-simplices

# Updates

In [7]:
data2 = data + 0.02 * np.random.randn(n,p)
plt.scatter(data2[:,0], data2[:,1]);

In [8]:
# compute pairwise distances
D2 = distance.squareform(distance.pdist(data2))

# Rips complex for full metric space
t0 = time.monotonic()
F2 = bats.LightRipsFiltration(bats.Matrix(D2), np.inf, 2)
t1 = time.monotonic()
print("construction time: {} sec.".format(t1-t0))

t0 = time.monotonic()
update_info = bats.UpdateInfoLightFiltration(F, F2)
t1 = time.monotonic()
print("compute UpdateInfo time: {} sec.".format(t1-t0))

t0 = time.monotonic()
R.update_filtration_general(update_info)
t1 = time.monotonic()
print("apply update time: {} sec.".format(t1-t0))

construction time: 0.04582071799999987 sec.
compute UpdateInfo time: 0.16435291499999982 sec.
apply update time: 0.04149440999999987 sec.


In [9]:
r_enc2 = np.min(np.max(D2, axis=0))

t0 = time.monotonic()
F_enc2 = bats.LightRipsFiltration(bats.Matrix(D2), r_enc2, 2)
t1 = time.monotonic()
print("construction time: {} sec.".format(t1-t0))

t0 = time.monotonic()
update_info_enc = bats.UpdateInfoLightFiltration(F_enc, F_enc2)
t1 = time.monotonic()
print("compute UpdateInfo time: {} sec.".format(t1-t0))

t0 = time.monotonic()
R_enc.update_filtration_general(update_info_enc)
t1 = time.monotonic()
print("apply update time: {} sec.".format(t1-t0))

construction time: 0.02074949299999984 sec.
compute UpdateInfo time: 0.07817569700000027 sec.
apply update time: 0.044906350999999844 sec.
